# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [2]:
data_dir = 'cats_dogs'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.RandomRotation(30),
                                       transforms.CenterCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor()])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [ ]:
model = models.densenet121(pretrained=True)
model

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [ ]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [ ]:
import time

In [2]:
torch.cuda.is_available()

False

In [ ]:
# for device in ['cpu', 'cuda']:
for device in ['cpu']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [3]:
## TODO: Use a pretrained model to classify the cat and dog images

model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

classifier = nn.Sequential(nn.Linear(1024,128),
                           nn.Dropout(.2),
                           nn.ReLU(),
                           #nn.Linear(256,128),
                           #nn.Dropout(.2),
                           #nn.ReLU(),
                           nn.Linear(128,8),
                           nn.Dropout(.2),
                           nn.ReLU(),
                           nn.Linear(8,2),
                           nn.LogSoftmax(dim=1))

model.classifier = classifier

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

/home/zelacerda/anaconda3/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [4]:
state_dict = torch.load('transfer_model.pth')
model.load_state_dict(state_dict)

In [6]:
epochs = 1

for e in range(epochs):
    running_loss = 0
    for i, (images, labels) in enumerate(trainloader):
        
        optimizer.zero_grad()
        #images = images.view(images.shape[0],-1)
        log_ps = model.forward(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        print("Batch: {}/{} .. Loss: {:.4f}".format(i, len(trainloader), loss.item()))
        running_loss += loss.item()
    
    print("Epoch: {}/{} /".format(e+1, epochs),
          "Train loss: {:.3f}".format(running_loss/len(trainloader)))

Batch: 0/352 .. Loss: 0.1028
Batch: 1/352 .. Loss: 0.1149
Batch: 2/352 .. Loss: 0.1231
Batch: 3/352 .. Loss: 0.1463
Batch: 4/352 .. Loss: 0.0308
Batch: 5/352 .. Loss: 0.1340
Batch: 6/352 .. Loss: 0.0440
Batch: 7/352 .. Loss: 0.2291
Batch: 8/352 .. Loss: 0.0447
Batch: 9/352 .. Loss: 0.0499
Batch: 10/352 .. Loss: 0.0553
Batch: 11/352 .. Loss: 0.1288
Batch: 12/352 .. Loss: 0.1013
Batch: 13/352 .. Loss: 0.1103
Batch: 14/352 .. Loss: 0.0742
Batch: 15/352 .. Loss: 0.1274
Batch: 16/352 .. Loss: 0.1095
Batch: 17/352 .. Loss: 0.0676
Batch: 18/352 .. Loss: 0.0295
Batch: 19/352 .. Loss: 0.1837
Batch: 20/352 .. Loss: 0.0212
Batch: 21/352 .. Loss: 0.0354
Batch: 22/352 .. Loss: 0.1196
Batch: 23/352 .. Loss: 0.0358
Batch: 24/352 .. Loss: 0.0383
Batch: 25/352 .. Loss: 0.0672
Batch: 26/352 .. Loss: 0.1006
Batch: 27/352 .. Loss: 0.0349
Batch: 28/352 .. Loss: 0.1407
Batch: 29/352 .. Loss: 0.0648
Batch: 30/352 .. Loss: 0.1218
Batch: 31/352 .. Loss: 0.0559
Batch: 32/352 .. Loss: 0.1665
Batch: 33/352 .. Los

Batch: 268/352 .. Loss: 0.0913
Batch: 269/352 .. Loss: 0.0233
Batch: 270/352 .. Loss: 0.0241
Batch: 271/352 .. Loss: 0.0938
Batch: 272/352 .. Loss: 0.0605
Batch: 273/352 .. Loss: 0.0468
Batch: 274/352 .. Loss: 0.0219
Batch: 275/352 .. Loss: 0.0964
Batch: 276/352 .. Loss: 0.1033
Batch: 277/352 .. Loss: 0.0505
Batch: 278/352 .. Loss: 0.1236
Batch: 279/352 .. Loss: 0.1515
Batch: 280/352 .. Loss: 0.0680
Batch: 281/352 .. Loss: 0.0150
Batch: 282/352 .. Loss: 0.1190
Batch: 283/352 .. Loss: 0.0615
Batch: 284/352 .. Loss: 0.1267
Batch: 285/352 .. Loss: 0.1129
Batch: 286/352 .. Loss: 0.0767
Batch: 287/352 .. Loss: 0.0558
Batch: 288/352 .. Loss: 0.0583
Batch: 289/352 .. Loss: 0.0630
Batch: 290/352 .. Loss: 0.0511
Batch: 291/352 .. Loss: 0.0520
Batch: 292/352 .. Loss: 0.0838
Batch: 293/352 .. Loss: 0.0432
Batch: 294/352 .. Loss: 0.0387
Batch: 295/352 .. Loss: 0.0140
Batch: 296/352 .. Loss: 0.1616
Batch: 297/352 .. Loss: 0.0286
Batch: 298/352 .. Loss: 0.0699
Batch: 299/352 .. Loss: 0.3128
Batch: 3

In [7]:
torch.save(model.state_dict(), 'transfer_model.pth')

In [10]:
accuracy = 0
test_loss = 0
model.eval()

with torch.no_grad():
    for inputs, labels in testloader:
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)
        test_loss += batch_loss.item()

        # Calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

model.train()

print("accuracy: {:.2%}".format(accuracy/len(testloader)))

accuracy: 90.43%
